In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
from bs4 import BeautifulSoup
import bs4

In [3]:
df_news = pd.read_csv('data/Macquarie_19900101-20220420.csv')

In [4]:
df_news

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,20220420,한국경제,김익환,대박 노리다 '700억' 물리더니 투자 스타일 확 바뀐 회사 [김익환의 컴퍼니워치],경제>금융_재테크,경제>증권_증시,경제>무역,NaN,NaN,NaN,"양홍석,양,이어룡,포메이션8,구본웅","미국,대주주,싱가포르,서울시,경기도","마음그룹,SC,LS그룹,예스코홀딩스,국민연,맥쿼리인프라,예스코,LS,대신증권,투자은행","700억,투자,스타일,회사,김익환,컴퍼니워치,LS,그룹,계열,도시가스,업체,예스코홀...","대신증권,예스코홀딩스,고배당주,맥쿼리인프라,도시가스,스타트업,부회장,700억,투자금...",LS그룹 계열의 도시가스업체 예스코홀딩스가 대신증권과 맥쿼리인프라를 비롯한 고배당주...,https://www.hankyung.com/economy/article/20220...,NaN
1,2.100101e+06,20220415,매일경제,신화,"미래에셋 리츠 ETF, 순자산 2000억 돌파",경제>증권_증시,경제>금융_재테크,경제>유통,NaN,NaN,NaN,NaN,NaN,"SK리츠,맥쿼리인프라,미래에셋,미래에셋자산운용,제이알글로벌리츠,ESR켄달스퀘어리츠,...","ETF,미래에셋,리츠,돌파,자산,미래에셋자산운용,TIGER,리츠부동산인프라,상장지수...","리츠부동산인프라,etf,2000억,원어치,미래에셋,인프라스트럭처,켄달스퀘어리츠,인컴...",미래에셋자산운용이 'TIGER 리츠부동산인프라' 상장지수펀드(ETF)의 순자산이 2...,http://news.mk.co.kr/newsRead.php?no=339384&ye...,NaN
2,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"김대기, 사외 감독이사 맡아 최소 6억 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,SK하이닉스,현대두산인프라코어,현대두산인프라...","김대기,사외,감독이사,수령,최소,관피아,논란,2015년,SK이노베이션,재직,한화생명...","내정자,사외이사,두산중공업,한화생명,맥쿼리인프라,두산인프라코어,sk이노베이션,공무원...",2015년부터 SK이노베이션 현대두산인프라코어 등서 재직\n한화생명에선 ‘고문’으로...,https://www.khan.co.kr/national/national-gener...,NaN
3,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"[단독]김대기, 민간 사외이사 지내며 ‘최소 6억4100만원’ 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,경제정의실천연합,현대두산인프라코어,맥쿼리한국...","김대기,민간,사외,이사,최소,수령,관피아,논란,김대기,대통령,비서실장,내정자,201...","내정자,사외이사,두산중공업,한화생명,sk이노베이션,이사회,두산인프라코어,맥쿼리인프라...",김대기 대통령 비서실장 내정자가 2015년부터 현재까지 민간 기업 사외 감독이사 활...,https://www.khan.co.kr/national/national-gener...,NaN
4,1.101001e+06,20220414,한겨레,박종오 기자,"김대기 비서실장, 바로 그 ‘맥쿼리’ 이사였다",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,노무현,이명박,김대기,윤증현,한승수,7천만원","인천김포고속도로,종로구,광주시,서울,9호선,통의동,서울지하철,서울시,부산신항,광주","에스케이이노베이션,인수위,맥쿼리인프라,대한상사중재원,서라벌도시가스,맥쿼리,두산중공업...","김대기,비서실장,맥쿼리,이사,김대기,대통령,비서실장,내정자,13일,민자,사업,투자,...","내정자,맥쿼리,이명박,맥쿼리인프라,민자사업,사외이사,한화생명,민자,윤증현,에스케이이...",김대기 대통령 비서실장 내정자가 13일 현재 민자사업 투자를 주력으로 하는 맥쿼리한...,http://www.hani.co.kr/arti/politics/politics_g...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,1.100901e+06,20060221,중앙일보,NaN,[브리핑] 인프라 펀드 상장 규모 11조원 넘을 듯,경제>증권_증시,국제>유럽_EU,경제>금융_재테크,NaN,NaN,NaN,NaN,"미국,환산,한국,호주,런던","신한금융그룹,매쿼리은행,메릴린치,투자은행,MKIF","11조,인프라,펀드,상장,규모,투자은행,호주,최대,투자,은행,매쿼리은행,신한금융그룹...","1조,한국,다음달,런던,메릴린치,mkif,매쿼리,호주,11조,11억,gdr,신한금융...",호주 최대 투자은행인 매쿼리은행과 신한금융그룹이 공동 설립한 인프라 펀드의 한국.런...,https://news.joins.com/article/2210809,예외
1023,2.100601e+06,20060209,한국경제,NaN,"CLSA,""인프라펀드 상장 관련 건설주 주목""",경제>금융_재테크,경제>증권_증시,경제>산업_기업,NaN,NaN,NaN,NaN,NaN,"CL,정부,크레디리요네(CLSA)증권,한경닷컴,PF","CLSA,인프라펀드,상장,상장,건설,주목,크레디리요네,CLSA,증권,맥쿼리인프라펀드...","건설주,크레디리요네,대림산업,현대산업개발,기투자,맥쿼리인프라펀드,기록중,인프라펀드,...",9일 크레디리요네(CLSA)증권은 맥쿼리인프라펀드(MKIF) 상장 추진으로 사모펀드...,NaN,NaN
1024,2.100601e+06,20060207,한국경제,NaN,"맥쿼리 인프라펀드, 韓ㆍ英 동시상장 추진",경제>증권_증시,경제>산업_기업,경제>국제경제,NaN,NaN,NaN,NaN,"한국,영국,항만,증시,런던","금융감독원,맥쿼리한국인프,맥쿼리,라투융자회사,MKIF","맥쿼리,인프라,펀드,추진,韓ㆍ英,동시,상장,도로,항만,사회간접자본,SOC,시설,투자...","맥쿼리,mkif,영국,런던,인프라펀드,사회간접자본,라투,감독원,라투융자회사,한국인프...",도로 항만 등 사회간접자본(SOC)시설에 투자하는 인프라펀드인 맥쿼리한국인프\n라투...,NaN,NaN
1025,1.100901e+06,20060207,중앙일보,NaN,[브리핑] 신한-맥쿼리 은행 펀드 한 영서 상장,경제>금융_재테크,경제>증권_증시,경제>국제경제,NaN,NaN,NaN,NaN,"서울,천안,호주,런던,인천공항","신한금융그룹,금융감독원,맥쿼리 은행,맥쿼리,맥쿼리코리아,MKIF","신한,맥쿼리,은행,펀드,영서,상장,금융감독원,신한금융그룹,호주,최대,투자,은행,맥쿼...","고속도로,신한금융그룹,호주,맥쿼리코리아,mkif,인프라스트럭처,천안,논산,감독원,맥...",금융감독원은 7일 신한금융그룹과 호주 최대 투자 은행인 맥쿼리 은행이 공동 설립한 ...,https://news.joins.com/article/2137057,예외


In [5]:
#Uncomment to export the original dataframe into the excel format
#df_news.to_excel("output.xlsx")

In [6]:
#Set as :20 to make a small sample
df_news = df_news.iloc[:]

In [7]:
pd.set_option('display.max_colwidth', None)
df_news['URL'].astype(str)
df_news['URL'].fillna('nan')

0                         https://www.hankyung.com/economy/article/202204208696i
1                          http://news.mk.co.kr/newsRead.php?no=339384&year=2022
2       https://www.khan.co.kr/national/national-general/article/202204142115005
3       https://www.khan.co.kr/national/national-general/article/202204141105001
4              http://www.hani.co.kr/arti/politics/politics_general/1038853.html
                                          ...                                   
1022                                      https://news.joins.com/article/2210809
1023                                                                         nan
1024                                                                         nan
1025                                      https://news.joins.com/article/2137057
1026                                                                         nan
Name: URL, Length: 1027, dtype: object

In [8]:
# Loop through URLs, scrap information, identify the main body
# URL이 주어졌을 경우 해당 매체 웹사이트로부터 뉴스기사전문 크롤링

#최종결과물에 대한 텍스트 전처리-불용어 제거
#반복적인 불용어에 대한 메뉴얼 수정 내용 추가 - 미디어 매채별 예외처리 등록
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
#    cleanr = re.compile('[^-가-힣a-zA-Z0-9/ ]')
    clean_text = re.sub(cleanr, '', raw_html)
    clean_text = clean_text.replace('window.jQuery || document.write("")', '')
    clean_text = clean_text.replace('Copyright ⓒ 동아일보 &amp; donga.com', '') #동아일보
    clean_text = clean_text.replace('좋아요 이미지좋아요슬퍼요 이미지슬퍼요화나요 이미지화나요후속기사 원해요 이미지후속기사 원해요기사추천 이미지기사추천공유 이미지공유공유하기닫기', '')
    clean_text = clean_text.replace('ⓒ 매일경제 &amp; mk.co.kr', '').replace('googletag.display("google_dfp_MC_250x250")', '').replace('googletag.display("google_dfp_MC_2x1,fluid")', '')
    clean_text = clean_text.replace('[', '').replace(']', '').replace('&amp', '').replace(', ■', '').replace('■ &lt', '').replace(';', '').replace('◆', '')
    return clean_text


def crawling_main_text(url):
        
        if type(url)==float:
            content = 'n/a'
        
        
        #조선일보 - 미완성
        elif 'chosun' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")
            content_obj = soup.find_all("div", class_="article-body")
            
            
            content = clean_html(str(content_obj))
        
        
        #중앙일보
        elif 'joins' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_body")
            content = clean_html(str(content_obj))
        
        
        #동아일보
        elif 'donga' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_txt")
            content = clean_html(str(content_obj))
        
        
        #한겨례신문
        elif 'hani' in url:
            url = urllib.request.Request(url)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
            
            content_obj = soup.find_all('div', class_="text")
            content = clean_html(str(content_obj))
        

        #경향신문
        elif 'khan' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('p', class_="content_text")
            content = clean_html(str(content_obj))
        
        
        #한국경제
        elif 'hankyung' in url:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
            url = urllib.request.Request(url, headers=headers)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
    
            content_obj = soup.find_all('div', id="articletxt")
            content = clean_html(str(content_obj))
        
        
        #매일경제
        elif 'mk' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
            soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

            content_obj = soup_alt.find_all('div', class_="art_txt")

            content = clean_html(str(content_obj))
    
    
        return content
                                     

In [ ]:
#For Loop을 통한 본문 크롤링
#데이터 프레임에 추가

df_news['main_content']='n/a'

for i, url in enumerate(df_news['URL']):
    print(url)
    print(type(url))
    try:
        main_body = crawling_main_text(url)
        print(main_body)
        df_news['main_content'].iloc[i] = main_body
    except ConnectionError:
        continue
    

https://www.hankyung.com/economy/article/202204208696i
<class 'str'>
 LS그룹 계열의 도시가스업체 예스코홀딩스가 대신증권과 맥쿼리인프라를 비롯한 고배당주를 500억원어치 넘게 사들인 것으로 나타났다. 이 회사는 해외 스타트업(신생 벤처기업)에 공격적으로 투자했다가 700억원 넘는 손실을 입었다. 해외 투자를 접고 비교적 안정적 수익을 내는 고배당주를 비롯한 안전자산 투자에 집중하는 배경으로 해석된다.20일 투자은행(IB)업계에 따르면 예스코홀딩스는 지난 15일 기준 대신증권 지분 1.76%(89만4533주)를 보유 중이다. 작년 12월에 대신증권 주식을 184억1700만원을 들여 매입한 이후 최근까지 보유 중이다. 예스코홀딩스는 대신증권과 함께 고배당주로 꼽히는 맥쿼리인프라 주식 243만425주도 작년 9월에 316억원에 매입했다. 이 회사가 대신증권과 맥쿼리인프라를 매입한 배경은 고배당으로 해석된다. 대신증권은 지난해 결산배당으로 주당 1400원을 결정했다. 이 회사의 배당수익률은 6.7%로 작년 상장사 평균(1.52%)을 크게 웃돈다. 예스코홀딩스는 배당수익으로 대신증권 배당수익으로만 올해 12억원을 받는다. 맥쿼리인프라는 지난해 배당수익률은 5.3%에 달한다.예스코홀딩스는 자회사 예스코를 통해 비교적 부침이 없는 도시가스사업을 하고 있다. 서울시와 경기도 141만가구에 배관을 통해 도시가스를 공급하는 업체로 연간 200억~300억 원 안팎의 당기순이익을 올렸다. 이 회사가 고배당주를 500억원어치 넘게 매입한 것은 뼈아픈 투자 실패와 관계가 깊다.예스코홀딩스는 2016년 음성인식 기술업체인 에바오토메이션에 116억원, 싱가포르 음식료 배송 스타트업인 어니스트비에 198억을 각각 투자했다. LS가(家) 장손 구본웅 마음그룹 대표가 운영하던 미국 벤처캐피털(VC)인 포메이션8이 투자한 회사들이다. 하지만 투자업체들이 모두 사업에 실패하면서 투자금을 모두 날렸다. 예스코홀딩스는 월드비즈니스렌더스(WLC)에

/Users/ohikendoit/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


신화 기자, 무단전재 및 재배포 금지	부동산인프라' 지수를 기초 지수로 추종한다.다. 순자산이 2000억원을 넘어선 것은 2019년 7월 상장한 이후 2년9개월 만이다. 물가가 오르면서 실물자산 가치가 상승할 가능성이 있고, 변동성 장세에 리츠의 안정성이 상대적으로 부각되며 자금이 몰린 것으로 풀이된다. 연초 이후 개인이 364억원어치를 순매수했으며, 보험과 기타법인도 각각 150억원어치, 135억원어치를 순매수했다. 이 ETF는 상장 리츠 중에서 롯데리츠, ESR켄달스퀘어리츠, SK리츠, 제이알글로벌리츠 등 시가총액이 2000억원 이상인 우량 리츠와 맥쿼리인프라와 같은 인프라스트럭처 투자 회사에 분산 투자한다. 보유 기간에 발생하는 인컴수익에 초점을 맞춘 상품이다. 소액으로도 부동산, 인프라 등에 투자할 수 있다는 특징이 있다.
https://www.khan.co.kr/national/national-general/article/202204142115005
<class 'str'>
2015년부터 SK이노베이션·현대두산인프라코어 등서 재직한화생명에선 ‘고문’으로…“모종의 ‘방패막’ 활용 위해 발탁”, 김대기 대통령비서실장 내정자(사진)가 2015년부터 현재까지 민간기업 사외·감독이사 활동으로만 최소 6억4100만원의 보수를 받은 것으로 파악됐다. 김 내정자는 2015년부터 2018년 초까지는 SK이노베이션과 두산인프라코어(현 현대두산인프라코어)에서 사외이사를 겸임했는데, 이 기간에만 총합 4억원이 넘는 보수를 받았다. 김 내정자는 현재 한화생명과 맥쿼리한국인프라투융자회사(맥쿼리인프라), 두산중공업에서 각각 고문 및 사외이사를 맡고 있다. , 14일 경향신문 취재를 종합하면 이명박 정부의 대통령비서실 경제수석, 대통령 정책실장을 끝으로 2013년 2월 공직을 떠난 김 내정자는 그해 한화생명 고문을 시작으로 SK이노베이션, 두산인프라코어, 두산중공업, 한화생명, 맥쿼리인프라에서 사외·감독이사를 맡았다. , 2015년 3월 SK이노베이션 사외이사에 선임된 김 내정자


http://news.mk.co.kr/newsRead.php?no=292701&year=2022
<class 'str'>
차창희 기자, 무단전재 및 재배포 금지	당금을 지급했는데 현 주가 기준 연 5.5%로 배당수익률이 높다. 맥쿼리인프라의 연간 주당 배당금은 2017년 540원, 2018년 622원, 2019년 700원, 2020년 720원, 2021년 750원으로 증가하고 있다. 회복) 기조에 따라 위축된 유료 도로 통행량이 회복되면서 통행료 수입도 증가하고 있다.다.
https://www.khan.co.kr/economy/economy-general/article/202203311551001
<class 'str'>
지난해 고위공직자들의 평균 재산이 16억2000만원으로 1년 전에 비해 1억6000만원 가량 증가한 것으로 나타났다. 재산이 증가한 원인으로는 부동산과 더불어 주식 가격 상승이 꼽혔다. 특히, 재산 증가액 1~2위를 기록한 공직자들은 보유하고 있던 대규모 비상장주식 가치가 급등한 덕을 봤다. 수십억원 규모의 주식을 보유한 ‘큰 손’ 공직자 중에는 국내 주식 외에 미국, 중국, 베트남 등 해외 주식에 투자한 경우도 눈에 띄었다. 수백억대 비상장 주식으로 재산 증가 , 정부공직자윤리위원회는 31일 관보에 2022년도 정기재산변동 신고사항을 공개했다. 이번 재산공개 대상 공직자 1978명 중 재산규모 1위(350억6768만원으로)와 더불어 증가액도 1위를 기록한 이강섭 법제처장은 본인, 배우자, 차녀의 증권 재산으로 257억4677만원을 신고했다. 1년 전인 29억6500만원에서 9배 가량 증가했다. , 이 처장 본인이 보유하고 있는 한건(1만4000주)과 한겨레신문(200주)의 비상장주식이 99억9046만원으로 지난해 평가액(7억7450만원)에 비해 92억원가량 불어난 결과다. 배우자가 보유한 한건(1만5000주), 씨앤팜(128주), 올바이오텍(16주) 등 비상장주식의 가치도 9억3138만원에서 121억3903만원으로 112억원 증가했다. 한

	올해 주요 자산 중 투자 수익률이 가장 좋았던 상품은 원유인 것으로 나타났다. 원유에 투자하는 상장지수펀드(ETF)는 작년 한 해 70% 가까이 떨어졌다가 올해는 정반대로 70% 오르는 반전을 이뤄냈다. 코로나19 사태 이후 글로벌 경기 급랭으로 마이너스 수익률을 기록한 리츠(REITs·부동산투자회사)도 꾸준한 성과를 내며 반등했다.코로나19 변이 바이러스의 등장, 스태그플레이션(경기침체 속 물가 상승) 우려, 미국 중앙은행(Fed)의 긴축 움직임 등 삼중고가 덮친 가운데서도 미국 주식은 20% 넘게 오르며 지난해에 이어 건재한 모습을 과시했다. 반면 투자자들의 안전자산 선호로 지난해 강세를 보였던 채권과 금은 주요 자산군 중 수익률 최하위권으로 떨어졌다.  원유 1등…희귀 광물도 초강세올해 주요 자산별 ETF 수익률을 집계한 결과 서부텍사스원유(WTI) 가격을 따르는 ETF가 최고 성과를 낸 것으로 나타났다. 세계 최대 원유 ETF인 ‘미국오일펀드’(USO)는 올 들어 이달 23일까지 69.3%의 수익률을 기록했다. 이 ETF는 지난해 수익률이 -67.7%였지만 올해는 가장 극적 반전을 이룬 자산에 올랐다.국제 유가는 올해 50% 이상 급등했다. 연초 배럴당 50달러 안팎에 형성됐던 WTI 선물 가격은 꾸준히 올라 10월엔 80달러 선을 돌파했다. 최근에는 주요 산유국이 석유 증산 방침을 유지하기로 한 데다 오미크론 변수 부각으로 수요 둔화 우려가 퍼지며 70달러 초반 선에서 거래되고 있다. 올해 유가 상승을 이끈 것은 세계 경제 회복 기대감이었다. 주요 투자은행(IB)들은 연말 코로나19 변이 바이러스 확산으로 수요가 영향을 받을 수 있다는 우려에도 내년 유가가 배럴당 100달러에 이를 것으로 전망하고 있다. JP모간은 브렌트유가 내년 배럴당 125달러를 넘어설 것으로 내다봤다. 현재 수준보다 60% 이상 오를 것이란 관측이다.구리 ETF도 올해 25.7% 성과를 내 상위권에 들었다. ‘미국구리인덱스펀드’(CPER)는 작년에 이어 올해도 20% 넘는 수익

...

맥쿼리인프라(14,000 +0.72%)가 1분기 안정적 실적 발표와 높은 배당수익률이 부각되면서 상승 중이다.2일 오전 9시 11분 현재 유가증권시장에서 맥쿼리인프라는 전날보다 200원(1.78%) 상승한 1만1450원에 거래되고 있다.라진성 키움증권 연구원은 "맥쿼리인프라는 1분기 운용수익이 1373억원으로 전년 대비 30.4%, 순이익은 1242억원으로 같은 기간 34.2% 늘었다"며 "지난해 2분기 서울-춘천 고속도로의 성공적 자금재조달 효과와 배당수익의 증가 때문"이라고 설명했다.이어 "최근 가파른 주가 상승이 부담스러울 수 있지만 여전히 시가배당률은 6.2%에 달한다"며 "오는 2020년과 2021년에도 주당분대금이 최소 700원은 유지할 수 있을 것"이라고 내다봤다.이송렬 한경닷컴 기자 yisr0203@hankyung.com
https://www.hankyung.com/article/201905024618L
<class 'str'>
 최근 애널리스트 분석의견- 안정적인 실적과 여전히 높은 배당수익률 - 키움증권, Outperform(유지)05월 02일 키움증권의 라진성 애널리스트는 맥쿼리인프라(14,000 +0.72%)에 대해 "BNP는 소폭 부진했지만, 자금재조달을 통해 중장기적으로 양호한 실적 성장과 재무구조 개선이 전망됩니다. 최근 가파른 주가 상승이 부담스러울 수 있지만 여전히 시가배당수익률은 6.2%에 달하는 상황입니다."이라고 분석하며, 투자의견 'Outperform(유지)', 목표주가 '12,000원'을 제시했다.한경로보뉴스이 기사는 한국경제신문과 금융 AI 전문기업 씽크풀이 공동 개발한 기사 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다.	
https://www.hankyung.com/article/201904308421L
<class 'str'>
 주체별 매매동향- 전일 외국인 대량 순매수지난 한달을 기준으로 보면 외국인이 124.6만주를 순매수했고, 개인들도 195.1만주를 순매수했다. 하지만 기관은 381.5만주를 순매도했다

	외국인 투자자는 25일 거래소에서 TIGER 200TR, 삼성SDI, 삼성전자 등을 중점적으로 사들인 것으로 나타났다.외국인 투자자의 순매수 상위 20개 종목은 TIGER 200TR, 삼성SDI, 삼성전자, 삼성물산, SK, 현대모비스, 롯데케미칼, 하나금융지주, 두산중공업, GS건설등이다.이중에 전기,전자 업종에 속한 종목이 3개 포함되어 있다.두산중공업, 현대로템, 맥쿼리인프라 등은 전일 대비 주가가 상승했고, TIGER 200TR, 삼성SDI, 삼성전자 등은 주가가 하락했다.03월25일거래소 외국인 순매수 상위 종목한경로보뉴스이 기사는 한국경제신문과 금융 AI 전문기업 씽크풀이 공동 개발한 기사 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다.	
http://news.hankyung.com/article/201903251322L
<class 'str'>
25일 맥쿼리인프라(14,000 +0.72%)는 장중 10,350원까지 오르며 52주 신고가를 갈아치우고, 현재 전일보다 100원(+0.98%) 오른 10,350원에 거래되고 있다. 주가의 흐름은 10,250원으로 개장해서 장중 한때 10,150원(-0.98%)까지 비교적 큰 폭으로 조정을 받았지만, 상승 반전해서 현재 10,350원(+0.98%)까지 상승하고 있다. 차트 분석- 전형적인 상승세, 단기·중기 이평선 정배열추세선으로 보면 현재 단기·중기 이평선이 정배열 상태로 전형적인 상승세를 보이고 있다.이 종목의 차트에서 최근에 최근 5일 매물대 돌파 등을 확인할 수 있다그래프맥쿼리인프라 차트 분석 주체별 매매동향- 전일 외국인 대량 순매도지난 한달을 기준으로 보면 외국인이 84.3만주를 순매수했고, 개인들도 141.8만주를 순매수했다. 하지만 기관은 272.6만주를 순매도했다. 같은 기간 이 종목의 거래 비중은 외국인과 기관이 각각 25.6%, 51.5%로 비중이 높다.한편 기관은 이 종목에 대해서 최근 9일 연속 263.8만주 순매도를 하고 있다. 더욱이 전일 외국인이 대량 순매도를 하고 있어 

	기관 투자자는 14일 거래소에서 두산, 두산중공업, 현대미포조선 등을 중점적으로 매도한 것으로 나타났다.기관 투자자의 순매도 상위 20개 종목은 두산, 두산중공업, 현대미포조선, KODEX 인버스, KODEX 코스닥150 레버리지, 현대중공업, 현대모비스, 삼성전기, 맥쿼리인프라, 두산인프라코어등이다.이중에 운수장비 업종에 속한 종목이 3개 포함되어 있다.KODEX 코스닥150 레버리지, 맥쿼리인프라, KT 등은 전일 대비 주가가 상승했고, 두산, 두산중공업, 현대미포조선 등은 주가가 하락했다.02월14일거래소 기관 순매도 상위 종목한경로보뉴스이 기사는 한국경제신문과 금융 AI 전문기업 씽크풀이 공동 개발한 기사 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다.	
http://news.hankyung.com/article/201902148138L
<class 'str'>
14일 맥쿼리인프라(14,000 +0.72%)는 장 초반 10,050원까지 오르며 지난 02월 13일 이후 1일 만에 52주 신고가를 새로 썼다. 오전 09시 35분 현재 전 거래일보다 0.5% 오른 10,050원에 거래되고 있다. 주가의 흐름을 살펴보면 9,990원으로 개장해서 장중 한때 9,980원(-0.2%)까지 살짝 밀렸지만, 상승 반전해서 현재 10,050원(+0.5%)까지 소폭 상승하고 있다. 최근 애널리스트 분석의견- 주주가치제고 진행중 - NH투자증권, BUY(유지)02월 01일 NH투자증권의 김동양, 박광삼 애널리스트는 맥쿼리인프라에 대해 "맥쿼리인프라(MKIF)의 2018년 운용수익, 당기순이익은 각각 2,616억원(9% y-y), 2,077억원(11% y-y, 이상 일회성 항목 제외 기준)운영중인 투자자산은 양호한 통행실적 기록. 가중평균 통행량 및 통행료수입은 전년동기대비 각 3.7%, 0.7% 증가"라고 분석하며, 투자의견 'BUY(유지)', 목표주가 '11,000원'을 제시했다.한경로보뉴스이 기사는 한국경제신문과 금융 AI 전문기업 씽크풀이 공동 개발한 기사 자동


http://news.mk.co.kr/newsRead.php?year=2019&no=41824
<class 'str'>
디지털뉴스국 고득관 기자, 무단전재 및 재배포 금지	프라에 투자한 주주들의 의견을 반영한 결과로 보수 구조를 더욱 명료하게 변경했다"며 "펀드 운용 안정성을 강화해 운용사는 장기적 주주가치 제고에 주력할 수 있을 것"이라고 기대했다.25%에서 0.85%로 낮아진다. 또 현재 해당 연도 목표 수익률인 연 8%를 초과하는 실적의 20% 수준에서 지급하도록 규정된 성과보수 항목은 삭제한다.
http://news.hankyung.com/article/2019012134426
<class 'str'>
맥쿼리인프라(14,000 +0.72%)가 운용사 기본보수를 내리고 성과보수를 폐지한다는 소식에 52주 신고가를 기록했다.21일 오전 9시15분 현재 유가증권시장에서 맥쿼리인프라는 전 거래일보다 130원(1.38%) 오른 9570원에 거래 중이다. 장중 한때는 9750원까지 올라 52주 신고가를 새로 썼다.앞서 맥쿼리인프라는 운용사 맥쿼리자산운용에 지급하는 기본보수 수준을 낮추고 성과보수는 폐지하기로 했다고 밝혔다. 보수 조정안에 따르면 기본보수 요율이 기존 맥쿼리인프라 시가총액의 1.10%??1.25%에서 0.85%로 낮아지고 성과보수 항목은 삭제된다.안혜원 한경닷컴 기자 anhw@hankyung.com
http://news.hankyung.com/article/201901213439L
<class 'str'>
21일 맥쿼리인프라(14,000 +0.72%)는 장 초반 9,750원까지 오르며 지난 01월 15일 이후 6일 만에 52주 신고가를 새로 썼다. 오전 09시 03분 현재 전 거래일보다 2.01% 오른 9,630원에 거래되고 있다. 주가의 흐름은 9,610원으로 개장해서 장중 한때 9,600원(+1.69%)까지 살짝 밀린 후 상승 반전해서 9,750원(+3.28%)까지 크게 올랐었는데, 현재는 9,630원(+2.01%)에서 머물고 있다. 주체별 매

디지털뉴스국, 무단전재 및 재배포 금지	룹 지주사 한진칼 지분을 9% 취득해 2대 주주로 올라서면서 최근 주목받고 있다. 상대로 주주권을 행사해 운용보수 인하를 끌어냈다. 개선과 배당 확대 등을 요구한다.경기 둔화 우려에 중장기물 위주로 채권 금리가 하락세를 보이면서 채권형 펀드의 양호한 성과로 이어졌다. 만기 1년 미만의 단기 투자상품인 초단기채권 펀드의 경우 2조8634억원이 들어왔다. 아시아퍼시픽(-10.84%), 신흥국(-10.46%), 중동아프리카(-9.02%), 중남미(-3.94%), 신흥유럽(-2.80%), 북미(-0.19%) 등 대부분 평가 손실을 내고 있다.줄줄이 손실을 낸 주식형 펀드와 달리 채권에 투자하는 펀드는 비교적 선전했다.
http://news.mk.co.kr/v2/economy/view.php?year=2018&no=784166
<class 'str'>

http://news.hankyung.com/article/201812132472L
<class 'str'>
	기관 투자자는 12일 거래소에서 삼성전기, 셀트리온, TIGER 200 등을 중점적으로 매도한 것으로 나타났다.기관 투자자의 순매도 상위 20개 종목은 삼성전기, 셀트리온, TIGER 200, KODEX 200, LG생활건강, 삼성SDI, 휠라코리아, 하나금융지주, 맥쿼리인프라, SK등이다.이중에 화학 업종에 속한 종목이 3개 포함되어 있다.삼성전기, 셀트리온, TIGER 200 등은 전일 대비 주가가 상승했고, LG생활건강, 휠라코리아, 현대로템 등은 주가가 하락했다.12월12일거래소 기관 순매도 상위 종목한경로보뉴스이 기사는 한국경제신문과 씽크풀이 공동 개발한 기사 자동생성 알고리즘에 의해 실시간으로 작성된 것입니다.- 기사오류문의 02-3277-9940	모바일한경 구독신청지면 구독신청ⓒ 한경닷컴, 무단전재 및 재배포 금지
http://news.hankyung.com/article/201812128240L
<class 'str'>
	기관 투자자는 11일 거래소에서 삼성

In [ ]:
#Export dataframe if necessary
df_news.to_excel("df_news_updated.xlsx")

  
  
  
  
  
## Reference Code for Individual Media Source  (Legacy; No Longer Updated)

### 한겨례

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

def contents_hankyoreh():
    
    #샘플 기사 URL
    list_url = "https://www.hani.co.kr/arti/politics/politics_general/1038853.html"
    
    url = urllib.request.Request(list_url)
    res = urllib.request.urlopen(url).read().decode("utf-8")
    #print(res)
    
    soup = BeautifulSoup(res, "html.parser")
    params1 = []
    params2 = []
    
    for link1, link2 in zip(soup.find_all('p', class_="date-time"), soup.find_all('div', class_="text")):
        params1.append(link1.get_text())
        params2.append(link2.get_text())
        
    for i1, i2 in zip(params1, params2):
        #print(i1.strip(), end=' ')
        print(i2.strip())
        
contents_hankyoreh()
    

### 한국경제

In [ ]:
#한국경제와 같은 경우 

import re
import urllib.request
from bs4 import BeautifulSoup

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def contents_hankyung():
    
    #샘플 기사 URL
    list_url = "https://www.hankyung.com/finance/article/2021122646691"
    
    url = urllib.request.Request(list_url, headers={'User-Agent': 'Custom'}) #
    res = urllib.request.urlopen(url).read().decode("utf-8")
    
    soup = BeautifulSoup(res, "html.parser")
    
    content_obj = soup.find_all('div', id="articletxt")
    content = cleanhtml(str(content_obj))
    return content
        

contents_hankyung()
    

### 경향신문

In [ ]:
#샘플 기사 URL
list_url = 'https://www.khan.co.kr/national/national-general/article/202204142115005'

def contents_kyunghyang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('p', class_="content_text")

    content = cleanhtml(str(content_obj))
    return content

contents_kyunghyang(list_url)

### 동아일보

In [ ]:
list_url = 'https://www.donga.com/news/article/all/20220404/112693737/1'

def contents_donga(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_donga(list_url)

### 중앙일보

In [ ]:
list_url = 'https://news.joins.com/article/2137057'

def contents_joongang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_body")

    content = cleanhtml(str(content_obj))
    return content

contents_joongang(list_url)

### 매일경제

In [ ]:
import bs4
list_url = 'https://www.mk.co.kr/news/stock/view/2022/04/339384/'

def contents_maeil(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
    soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

    content_obj = soup_alt.find_all('div', class_="art_txt")

    content = clean_html(str(content_obj))
    return content

contents_maeil(list_url)

### 조선일보

In [ ]:
!pip install selenium

In [ ]:
#조선일보 같은 경우 셀레니움 패키지를 사용

import re
import requests
from bs4 import BeautifulSoup
import urllib.request
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(ChromeDriverManager().install())

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content_obj = []

#Legacy Component
def contents_chosun(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")
    print(soup)
    content_obj = soup.find_all('p')
    print(content_obj)
    
    content = clean_html(str(content_obj))
    
    if soup.select_one('#text'):
        soup.dl.extract()
        news_article = soup.select_one('#article').text
        print(news_article)
    
#    for tag in soup.select('#text'):
#        print(tag.text)
#        write_ws.append([tag.text])
    
#    content = clean_html(str(content_obj))
    return content

contents_chosun(list_url)

In [ ]:
#셀레니윰을 통한 조선일보 크롤링 접근

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

import pandas as pd
from selenium.common.exceptions import NoSuchElementException

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content = []


#Selenium Component
driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)
driver.get(list_url)

driver.implicitly_wait(time_to_wait=5)
print(driver)

#more = driver.find_element_by_id('load-more-stories')

#for _ in range(120):
#    more.click()
#    time.sleep(1)
    
#selector = '#artwrapper > div > figure > div > div > div'
#a = driver.find_element(selector)

try:
    content_section = driver.find_element('//*[@id="fusion-app"]/div[1]/div[2]/div/section/article/section')
    p = content_section.find_elements('p')
    temp = []
    for item in p:
        temp.append(item.text)
    content.append('\n'.join(temp))
except:
    content.append(pd.NA)

driver.close()

## 셀레니움을 통한 조선일보 크롤링 구현

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

In [ ]:
driver = webdriver.Chrome(executable_path='/Users/ohikendoit/chromedriver', options=options)
try:
    driver.get("https://www.google.com/")
except Exception as e:
    print(e.message)

In [ ]:
driver.get("https://www.google.com/")

In [ ]:
start_date = '20200501'
end_date = '20200502'
paper = 1023

num = 1
while True:
    driver.get(f'https://search.naver.com/search.naver?where=news&query=%EC%A7%80%EC%97%AD%ED%99%94%ED%8F%90&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date[0:4]}.{start_date[4:6]}.{start_date[6:]}&de={end_date[0:4]}.{end_date[4:6]}.{end_date[6:]}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={paper}&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date},a:all&start={num}')
    time.sleep(3)
    
    try:
        ul = driver.find_element(By.CSS_SELECTOR, '#main_pack > section > div > div.group_news > ul')
    except NoSuchElementException:
        print("Not exist paper")
        break
    lis = ul.find_elements(By.CLASS_NAME, 'li') #The previous approach has been depreciated.
    
    check = False
    
    for li in lis:
        tmp = li.get_attribute('id')
        tmp_url = driver.find_element(By.CSS_SELECTOR, f'#{tmp} > div > div > a').get_attribute("href")
        if tmp_url in url_list:
            check = True
            break
        url_list.append(tmp_url)
        
    if check:
        break
        
    num += 10

In [ ]:
url_list = ['https://biz.chosun.com/site/data/html_dir/2020/04/23/2020042302274.html?utm_source=bigkinds&utm_medium=original&utm_campaign=biz']

In [ ]:
for url in url_list:
    driver.get(url)
    time.sleep(1)
    content = driver.find_element_by_xpath('//*[@id="fusion-app"]/div[1]/div[2]/div/section/article/section').text
    print(content)
    
    print('='*30)
    print('='*30)